In [ ]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib as mplib
import matplotlib.pyplot as plt
import matplotlib.image as img
import itertools
from keras import layers, models
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD
from collections import Counter
from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('./hmnist_28_28_RGB.csv')
print(data.shape)

In [ ]:
Y = data['label']
print(type(Y))
print(Counter(Y))
nlabels = Y.value_counts().size
print(nlabels)

In [ ]:
X = data.drop(['label'],axis=1)
X = X/255

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
smote = SMOTE(random_state = 50)
X_resample, Y_resample = smote.fit_resample(X, Y)

In [ ]:
print(X_resample.shape)
print(Y_resample.shape)

In [ ]:
unique, counts = np.unique(Y_resample, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
X_resample = X_resample.values.reshape(-1,28,28,3)

In [ ]:
Y_resample = tf.keras.utils.to_categorical(Y_resample, num_classes=nlabels)
print(Y_resample.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_resample, Y_resample, test_size=0.1, random_state=1)

In [ ]:
print(Y_train.shape)
print(X_train.shape)
print(Y_test.shape)
print(X_test.shape)

In [ ]:
#CNN Architecture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 ->
                           #Flatten -> Dense -> Dropout -> Out
model = models.Sequential()


model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu', input_shape=(28,28,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.1))


model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu', input_shape = (28,28,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.1))


model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu', input_shape = (28,28,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.1))


model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(nlabels,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
augdata = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,
)  # randomly flip images

augdata.fit(X_train)

In [ ]:
unique, counts = np.unique(Y_resample, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
data1 = model.fit(
    X_train,
    Y_train,
    epochs=10,
    verbose = 1,
    # validation_data = (X_val, Y_val),
#     steps_per_epoch = X_train.shape[0] // 128,
)

In [ ]:
# Plot the loss and accuracy curves for training and validation
fig, ax = plt.subplots(2,1)
ax[0].plot(data1.history['loss'], color='b', label="Training loss")
# ax[0].plot(data1.history['val_loss'], color='r', label="Validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)


ax[1].plot(data1.history['accuracy'], color='b', label="Training accuracy")
# ax[1].plot(data1.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)


In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
# Look at confusion matrix


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)


    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Predict the values from the validation dataset
Y_pred = model.predict(X_test)
# print(Y_pred)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred,axis = 1)
# print()
# print(Y_pred_classes)
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_test,axis = 1)
# print()
# print(Y_true)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10))


In [ ]:
# Display some error results


# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)


Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_test[errors]


def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]))
            ax[row,col].set_title(" Predicted :{} True :{}".format(pred_errors[error],obs_errors[error]))
            n += 1


# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)


# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))


# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors


# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)


# Top 6 errors
most_important_errors = sorted_dela_errors[-6:]
# print(X_val_errors[most_important_errors[5]].shape)

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)
